In [ ]:
# Animal Image Classification Project
# Objective: Build a system that can identify animals in given images using Neural Networks and Transfer Learning
# Dataset: 15 animal classes with 30 images each (450 total images)

# Install required packages
!pip install tensorflow opencv-python matplotlib seaborn scikit-learn pillow -q

# Import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('default')
sns.set_palette("husl")

print("🐾 Animal Classification Project")
print("TensorFlow version:", tf.__version__)




In [ ]:
# ==================== DATA SETUP ====================

# Upload and extract the dataset ZIP file
from google.colab import files
import zipfile

print("📁 Upload your dataset folder here:")
uploaded = files.upload()

# Extract the ZIP file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"📦 Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print("✅ Dataset extracted successfully!")
        break

# Set dataset path
data_path = 'Animal Classification/dataset'

# Configuration
IMG_SIZE = (160, 160)
BATCH_SIZE = 32
EPOCHS = 25
NUM_CLASSES = 15

# Animal classes
CLASSES = ['Bear', 'Bird', 'Cat', 'Cow', 'Deer', 'Dog', 'Dolphin',
          'Elephant', 'Giraffe', 'Horse', 'Kangaroo', 'Lion', 'Panda', 'Tiger', 'Zebra']


In [ ]:
# ==================== DATA EXPLORATION ====================

def explore_dataset(data_path):
    """Explore the dataset structure and statistics"""

    if not os.path.exists(data_path):
        print("⚠️  Dataset path not found!")
        return None

    print("📊 Dataset Overview")
    print("=" * 40)

    class_counts = {}
    total_images = 0

    for class_name in CLASSES:
        class_path = os.path.join(data_path, class_name)
        if os.path.exists(class_path):
            count = len([f for f in os.listdir(class_path)
                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            class_counts[class_name] = count
            total_images += count
        else:
            class_counts[class_name] = 0

    df = pd.DataFrame(list(class_counts.items()), columns=['Animal', 'Count'])
    print(f"Total Images: {total_images}")
    print(f"Average per class: {total_images // len(CLASSES)}")

    # Visualization
    plt.figure(figsize=(12, 5))
    bars = plt.bar(df['Animal'], df['Count'], color=sns.color_palette("husl", len(CLASSES)))
    plt.title('🐾 Dataset Distribution', fontsize=14, fontweight='bold')
    plt.xlabel('Animal Classes')
    plt.ylabel('Number of Images')
    plt.xticks(rotation=45, ha='right')

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}', ha='center', va='bottom', fontsize=8)

    plt.tight_layout()
    plt.show()

    return df

# Explore dataset
dataset_stats = explore_dataset(data_path)


In [ ]:
# ==================== DATA PREPROCESSING ====================

def create_data_generators():
    """Create data generators with augmentation"""

    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.1,
        brightness_range=[0.8, 1.2],
        validation_split=0.2,
        fill_mode='nearest'
    )

    # Training generator
    train_generator = train_datagen.flow_from_directory(
        data_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    # Validation generator
    validation_generator = train_datagen.flow_from_directory(
        data_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )

    return train_generator, validation_generator

# Create data generators
print("🔄 Creating data generators...")
train_gen, val_gen = create_data_generators()
print(f"✅ Training samples: {train_gen.samples}")
print(f"✅ Validation samples: {val_gen.samples}")

In [ ]:
# ==================== MODEL BUILDING ====================

def create_model():
    """Create transfer learning model using MobileNetV2"""

    print("🧠 Building model with MobileNetV2...")

    # Load pre-trained MobileNetV2
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(*IMG_SIZE, 3),
        alpha=1.0
    )

    # Freeze base model initially
    base_model.trainable = False

    # Add custom classification head
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("✅ Model created successfully!")
    print(f"Total parameters: {model.count_params():,}")

    return model, base_model

# Create model
model, base_model = create_model()

# Display model architecture
model.summary()

In [ ]:
# ==================== TRAINING ====================

def create_callbacks():
    """Create training callbacks"""

    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.3,
            patience=3,
            min_lr=1e-7,
            verbose=1
        )
    ]

    return callbacks

def train_model(model, train_gen, val_gen, base_model, epochs=EPOCHS):
    """Train model using transfer learning approach"""

    print("🚀 Starting Training!")
    print("=" * 40)

    callbacks = create_callbacks()

    # Phase 1: Train classifier head
    print("📚 Phase 1: Training classifier head...")

    history_1 = model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // BATCH_SIZE,
        validation_data=val_gen,
        validation_steps=val_gen.samples // BATCH_SIZE,
        epochs=epochs // 2,
        callbacks=callbacks,
        verbose=1
    )

    # Phase 2: Fine-tuning
    print("\n🎯 Phase 2: Fine-tuning model...")

    # Unfreeze top layers of base model
    base_model.trainable = True

    # Fine-tune from this layer onwards
    fine_tune_at = len(base_model.layers) - 20

    # Freeze all the layers before fine_tune_at
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # Recompile with lower learning rate
    model.compile(
        optimizer=Adam(learning_rate=0.0001/10),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    history_2 = model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // BATCH_SIZE,
        validation_data=val_gen,
        validation_steps=val_gen.samples // BATCH_SIZE,
        epochs=epochs - len(history_1.history['loss']),
        callbacks=callbacks,
        initial_epoch=len(history_1.history['loss']),
        verbose=1
    )

    # Combine histories
    history = {}
    for key in history_1.history.keys():
        history[key] = history_1.history[key] + history_2.history[key]

    return history

# Train the model
import time
start_time = time.time()

history = train_model(model, train_gen, val_gen, base_model)

end_time = time.time()
training_time = (end_time - start_time) / 60
print(f"🎉 Training completed in {training_time:.1f} minutes!")


In [ ]:
# ==================== RESULTS VISUALIZATION ====================

def plot_training_results(history):
    """Plot training results"""

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # Accuracy
    axes[0].plot(history['accuracy'], 'bo-', label='Training Accuracy', linewidth=2)
    axes[0].plot(history['val_accuracy'], 'ro-', label='Validation Accuracy', linewidth=2)
    axes[0].set_title('📈 Model Accuracy', fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Loss
    axes[1].plot(history['loss'], 'bo-', label='Training Loss', linewidth=2)
    axes[1].plot(history['val_loss'], 'ro-', label='Validation Loss', linewidth=2)
    axes[1].set_title('📉 Model Loss', fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

def evaluate_model(model, val_gen):
    """Evaluate model performance"""

    print("🎯 Evaluating Model Performance...")

    # Get predictions
    val_gen.reset()
    predictions = model.predict(val_gen, verbose=1)
    pred_classes = np.argmax(predictions, axis=1)
    true_classes = val_gen.classes
    class_names = list(val_gen.class_indices.keys())

    # Classification report
    print("\n📊 Classification Report:")
    print("=" * 50)
    print(classification_report(true_classes, pred_classes, target_names=class_names))

    # Confusion matrix
    cm = confusion_matrix(true_classes, pred_classes)

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('🎯 Confusion Matrix', fontsize=14, fontweight='bold')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

    return predictions, pred_classes, true_classes

def show_sample_predictions(model, val_gen, num_samples=8):
    """Display sample predictions"""

    val_gen.reset()
    images, labels = next(val_gen)
    predictions = model.predict(images)

    class_names = list(val_gen.class_indices.keys())

    plt.figure(figsize=(15, 8))

    for i in range(min(num_samples, len(images))):
        plt.subplot(2, 4, i + 1)
        plt.imshow(images[i])

        true_label = class_names[np.argmax(labels[i])]
        pred_label = class_names[np.argmax(predictions[i])]
        confidence = np.max(predictions[i]) * 100

        color = 'green' if true_label == pred_label else 'red'
        plt.title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.1f}%',
                 color=color, fontsize=9, fontweight='bold')
        plt.axis('off')

    plt.suptitle('🔍 Sample Predictions', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Visualize training results
plot_training_results(history)

# Evaluate model performance
predictions, pred_classes, true_classes = evaluate_model(model, val_gen)

# Show sample predictions
show_sample_predictions(model, val_gen)

In [ ]:
# ==================== FINAL RESULTS ====================

print("\n" + "="*60)
print("🏆 MODEL PERFORMANCE SUMMARY")
print("="*60)

best_val_acc = max(history['val_accuracy'])
best_val_loss = min(history['val_loss'])
final_accuracy = np.mean(pred_classes == true_classes)

print(f"🎯 Final Accuracy: {final_accuracy:.3f} ({final_accuracy*100:.1f}%)")
print(f"🏆 Best Validation Accuracy: {best_val_acc:.3f} ({best_val_acc*100:.1f}%)")
print(f"📉 Best Validation Loss: {best_val_loss:.3f}")
print(f"📊 Total Training Epochs: {len(history['loss'])}")
print(f"⏱️  Training Time: {training_time:.1f} minutes")
print("="*60)

if final_accuracy > 0.8:
    print("🎉 Model achieved excellent performance!")
elif final_accuracy > 0.6:
    print("✅ Model shows good performance!")
else:
    print("⚠️  Model performance could be improved.")

print("\n🐾 Animal Classification Project Complete!")
print("The model successfully classifies 15 different animal species using transfer learning.")